In [6]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np


Batch_Size = 250
num_epochs = 100
learning_rate = 1e-3
samples = 50000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    r = np.random.choice(np.arange(1, 4), p=[0.3,0.3,0.4])
    if r==1:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data1.append(np.asarray(im2_pos1))
    elif r==2:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2_pos1))
    elif r==3:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        data1.append(np.asarray(im2_pos1))
        




for i in range(samples):
    
    r = np.random.choice(np.arange(1, 4), p=[0.3,0.3,0.4])

    if r==1:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw = ImageDraw.Draw(im2_pos2)
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2_pos2))
    elif r==2:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw = ImageDraw.Draw(im2_pos2)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data2.append(np.asarray(im2_pos2))
    elif r==3:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        data2.append(np.asarray(im2_pos2))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (28, 28), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 1, 2))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 1, 2)) 


print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True), # b, 8, 2, 2
            nn.Conv2d(8,4,3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(4,2,3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1,2)
        x = self.fc(x)
        return x

class auto_decoder(nn.Module):
    def __init__(self):
        super(auto_decoder,self).__init__()
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 3,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 3,padding=1)
        
    def forward(self, x):
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x
    

model = autoencoder()
model_1 = auto_decoder()

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

criterion = nn.MSELoss()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=learning_rate,weight_decay=1e-5)


for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))
    print("b:", model(b.reshape(1,3,28,28)))



(50000, 3, 28, 28)
epoch [1/100], loss:-0.0006
x:  tensor([[-0.1178]], grad_fn=<AddmmBackward>)
y: tensor([[-0.0619]], grad_fn=<AddmmBackward>)
b: tensor([[-0.1897]], grad_fn=<AddmmBackward>)
epoch [2/100], loss:-0.0005
x:  tensor([[0.0122]], grad_fn=<AddmmBackward>)
y: tensor([[0.0708]], grad_fn=<AddmmBackward>)
b: tensor([[-0.0256]], grad_fn=<AddmmBackward>)
epoch [3/100], loss:0.0023
x:  tensor([[0.9595]], grad_fn=<AddmmBackward>)
y: tensor([[1.0342]], grad_fn=<AddmmBackward>)
b: tensor([[0.7957]], grad_fn=<AddmmBackward>)
epoch [4/100], loss:0.0024
x:  tensor([[1.0903]], grad_fn=<AddmmBackward>)
y: tensor([[0.9659]], grad_fn=<AddmmBackward>)
b: tensor([[0.9630]], grad_fn=<AddmmBackward>)
epoch [5/100], loss:0.0040
x:  tensor([[1.3406]], grad_fn=<AddmmBackward>)
y: tensor([[1.0172]], grad_fn=<AddmmBackward>)
b: tensor([[0.8534]], grad_fn=<AddmmBackward>)
epoch [6/100], loss:-0.0001
x:  tensor([[1.5707]], grad_fn=<AddmmBackward>)
y: tensor([[1.0073]], grad_fn=<AddmmBackward>)
b: tens

epoch [49/100], loss:-0.0129
x:  tensor([[0.8130]], grad_fn=<AddmmBackward>)
y: tensor([[1.4399]], grad_fn=<AddmmBackward>)
b: tensor([[0.7421]], grad_fn=<AddmmBackward>)
epoch [50/100], loss:-0.0423
x:  tensor([[1.6612]], grad_fn=<AddmmBackward>)
y: tensor([[1.5082]], grad_fn=<AddmmBackward>)
b: tensor([[1.3881]], grad_fn=<AddmmBackward>)
epoch [51/100], loss:-0.0252
x:  tensor([[1.6646]], grad_fn=<AddmmBackward>)
y: tensor([[1.6952]], grad_fn=<AddmmBackward>)
b: tensor([[1.3141]], grad_fn=<AddmmBackward>)
epoch [52/100], loss:-0.0098
x:  tensor([[2.1314]], grad_fn=<AddmmBackward>)
y: tensor([[2.2378]], grad_fn=<AddmmBackward>)
b: tensor([[1.6574]], grad_fn=<AddmmBackward>)
epoch [53/100], loss:-0.0375
x:  tensor([[2.9710]], grad_fn=<AddmmBackward>)
y: tensor([[2.9385]], grad_fn=<AddmmBackward>)
b: tensor([[2.4008]], grad_fn=<AddmmBackward>)
epoch [54/100], loss:-0.0346
x:  tensor([[2.2004]], grad_fn=<AddmmBackward>)
y: tensor([[2.8985]], grad_fn=<AddmmBackward>)
b: tensor([[1.6080]],

epoch [97/100], loss:-0.0471
x:  tensor([[2.5370]], grad_fn=<AddmmBackward>)
y: tensor([[1.8830]], grad_fn=<AddmmBackward>)
b: tensor([[1.7972]], grad_fn=<AddmmBackward>)
epoch [98/100], loss:-0.0261
x:  tensor([[0.8518]], grad_fn=<AddmmBackward>)
y: tensor([[1.2487]], grad_fn=<AddmmBackward>)
b: tensor([[1.2751]], grad_fn=<AddmmBackward>)
epoch [99/100], loss:-0.0428
x:  tensor([[2.3403]], grad_fn=<AddmmBackward>)
y: tensor([[1.0576]], grad_fn=<AddmmBackward>)
b: tensor([[1.5498]], grad_fn=<AddmmBackward>)
epoch [100/100], loss:-0.0625
x:  tensor([[1.3192]], grad_fn=<AddmmBackward>)
y: tensor([[1.4381]], grad_fn=<AddmmBackward>)
b: tensor([[1.4955]], grad_fn=<AddmmBackward>)


In [7]:
for i in range(20):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2_n = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2_n)
    b = np.array(b, dtype=np.float32)
    b = np.transpose(b,(2,1,0))
    b = torch.from_numpy(b)
    im2_test = Image.fromarray(noisy,'RGB')
    draw1 = ImageDraw.Draw(im2_test)
    draw1.rectangle([5, 9, 8, 12], fill='red', outline='red')
    z = np.asarray(im2_test)
    z = np.array(z, dtype=np.float32)
    z = torch.from_numpy(z)
    print("b:", model(b.reshape(1,3,28,28)))
    print("z: ", model(z.reshape(1,3,28,28)))
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("  ")
    p = Image.fromarray(data1[i])
    #p.show()
    q = Image.fromarray(data2[i])
    #q.show()

b: tensor([[1.1789]], grad_fn=<AddmmBackward>)
z:  tensor([[1.5266]], grad_fn=<AddmmBackward>)
x:  tensor([[1.3192]], grad_fn=<AddmmBackward>)
y: tensor([[1.4381]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[1.1660]], grad_fn=<AddmmBackward>)
z:  tensor([[2.0903]], grad_fn=<AddmmBackward>)
x:  tensor([[1.8812]], grad_fn=<AddmmBackward>)
y: tensor([[1.7965]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[1.7243]], grad_fn=<AddmmBackward>)
z:  tensor([[1.6436]], grad_fn=<AddmmBackward>)
x:  tensor([[1.5179]], grad_fn=<AddmmBackward>)
y: tensor([[1.4492]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[1.8237]], grad_fn=<AddmmBackward>)
z:  tensor([[1.6614]], grad_fn=<AddmmBackward>)
x:  tensor([[1.7356]], grad_fn=<AddmmBackward>)
y: tensor([[0.8454]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%